In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
from bs4 import BeautifulSoup
import requests 

In [3]:
url='https://groww.in/mutual-funds/filter?q=&fundSize=&pageNo=0&sortBy=0'
page=requests.get(url)
page

<Response [200]>

In [4]:
soup=BeautifulSoup(page.content,'html.parser')

In [6]:
# html section containing each fund information in first page
fund=soup.find_all('tr',class_="f22Card")

In [7]:
name=[]    # name of fund
rating=[]  # performance rating of fund by experts 
return_1yr=[]   # last 1 yr return
return_3yr=[]    # last 3 yr return
return_5yr=[]    # last 5 yr return
risk_type=[]     # degree of risk
category=[]      # fund_category

for i in fund:
    name.append(i.find('div',class_="fs14 clrText fw500 f22LH34 f22Mb4 truncate").text.strip())
    return_1yr.append(i.find_all('div',class_="fs14 clrText fw500 center-align f22Mb4")[0].text.strip())
    return_3yr.append(i.find_all('div',class_="fs14 clrText fw500 center-align f22Mb4")[1].text.strip())
    return_5yr.append(i.find_all('div',class_="fs14 clrText fw500 center-align f22Mb4")[2].text.strip())
    risk_type.append(i.find('div',class_="fs12 fw500 clrSubText f22Ls2").text.strip())
    category.append(i.find('div',class_="fs12 fw500clrSubText f22Ls2").text.strip())
    try: 
          rating.append(i.find('div',class_="fs12 fw500 clrSubText f22Ls2 valign-wrapper").text.strip())
    except:
          rating.append(np.nan)
            
# creating a Dataframe out of first page of results            
            
d={'Mutual Fund Name':name,'fund_rating':rating,'return_1yr':return_1yr,'return_3yr':return_3yr,'return_5yr':return_5yr,
   'risk_type':risk_type,'category':category}
df=pd.DataFrame(d)

In [8]:
df

,Mutual Fund Name,fund_rating,return_1yr,return_3yr,return_5yr,risk_type,category
0,Quant Small Cap Fund Direct Plan Growth,4,9.98%,54.37%,22.73%,Very High Risk,Equity
1,Canara Robeco Small Cap Fund Direct Growth,5,20.09%,44.48%,NA%,Very High Risk,Equity
2,PGIM India Midcap Opportunities Fund Direct Gr...,5,11.02%,44.22%,21.49%,Very High Risk,Equity
3,Bank of India Small Cap Fund Direct Growth,5,13.16%,44.13%,NA%,Very High Risk,Equity
4,Quant Tax Plan Direct Growth,5,17.43%,43.88%,23.76%,Very High Risk,Equity
5,Quant Infrastructure Fund Direct Growth,5,25.78%,43.74%,23.91%,Very High Risk,Equity
6,Quant Mid Cap Fund Direct Growth,5,20.43%,40.20%,22.03%,Very High Risk,Equity
7,Quant Flexi Cap Fund Direct Growth,NaN,8.31%,39.82%,19.71%,Very High Risk,Equity
8,Quant Active Fund Direct Growth,NaN,12.41%,39.22%,23.40%,Very High Risk,Equity
9,Kotak Small Cap Fund Direct Growth,4,10.06%,39.13%,19.66%,Very High Risk,Equity


# Compiling mutual fund data from all pages of Groww Investment platform

In [9]:
final=pd.DataFrame()

for j in range(73):
    url='https://groww.in/mutual-funds/filter?q=&fundSize=&pageNo={}&sortBy=0'.format(j)
    page=requests.get(url)
    soup=BeautifulSoup(page.content,'html.parser')
 
  #  fund containers
    fund=soup.find_all('tr',class_="f22Card")

  #scraping all field from each section
    name=[]
    rating=[]
    return_1yr=[]
    return_3yr=[]
    return_5yr=[]
    risk_type=[]
    category=[]

    for i in fund:
        name.append(i.find('div',class_="fs14 clrText fw500 f22LH34 f22Mb4 truncate").text.strip())
        return_1yr.append(i.find_all('div',class_="fs14 clrText fw500 center-align f22Mb4")[0].text.strip())
        return_3yr.append(i.find_all('div',class_="fs14 clrText fw500 center-align f22Mb4")[1].text.strip())
        return_5yr.append(i.find_all('div',class_="fs14 clrText fw500 center-align f22Mb4")[2].text.strip())
        risk_type.append(i.find('div',class_="fs12 fw500 clrSubText f22Ls2").text.strip())
        category.append(i.find('div',class_="fs12 fw500clrSubText f22Ls2").text.strip())
        try: 
              rating.append(i.find('div',class_="fs12 fw500 clrSubText f22Ls2 valign-wrapper").text.strip())
        except:
              rating.append(np.nan)

    d={'Mutual Fund Name':name,'fund_rating':rating,'return_1yr':return_1yr,
      'return_3yr':return_3yr,'return_5yr':return_5yr,
      'risk_type':risk_type,'category':category} 
    
    df=pd.DataFrame(d)

    #combining dataframes from each page
    final=final.append(df,ignore_index=True)  

In [10]:
final.shape

(1088, 7)

In [11]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1088 entries, 0 to 1087
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Mutual Fund Name  1088 non-null   object
 1   fund_rating       738 non-null    object
 2   return_1yr        1088 non-null   object
 3   return_3yr        1088 non-null   object
 4   return_5yr        1088 non-null   object
 5   risk_type         1088 non-null   object
 6   category          1088 non-null   object
dtypes: object(7)
memory usage: 59.6+ KB


In [12]:
final.sample(3)

,Mutual Fund Name,fund_rating,return_1yr,return_3yr,return_5yr,risk_type,category
1080,HDFC Multi Cap Fund Direct Growth,NaN,NA%,NA%,NA%,Very High Risk,Equity
318,HDFC Top 100 Fund Direct Plan Growth,2,9.64%,16.89%,11.83%,Very High Risk,Equity
141,PGIM India ELSS Tax Saver Fund Direct Growth,4,8.28%,22.57%,14.16%,Very High Risk,Equity


In [13]:
final['return_1yr']=final['return_1yr'].apply(lambda x:x.replace("%",''))
final['return_3yr']=final['return_3yr'].apply(lambda x:x.replace("%",''))
final['return_5yr']=final['return_5yr'].apply(lambda x:x.replace("%",''))

In [14]:
final['AMC_name']=final['Mutual Fund Name'].apply(lambda x:x.split()[0])

In [15]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1088 entries, 0 to 1087
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Mutual Fund Name  1088 non-null   object
 1   fund_rating       738 non-null    object
 2   return_1yr        1088 non-null   object
 3   return_3yr        1088 non-null   object
 4   return_5yr        1088 non-null   object
 5   risk_type         1088 non-null   object
 6   category          1088 non-null   object
 7   AMC_name          1088 non-null   object
dtypes: object(8)
memory usage: 68.1+ KB


In [16]:
final['fund_rating'] = pd.to_numeric(final['fund_rating'], errors='coerce')

In [17]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1088 entries, 0 to 1087
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Mutual Fund Name  1088 non-null   object 
 1   fund_rating       738 non-null    float64
 2   return_1yr        1088 non-null   object 
 3   return_3yr        1088 non-null   object 
 4   return_5yr        1088 non-null   object 
 5   risk_type         1088 non-null   object 
 6   category          1088 non-null   object 
 7   AMC_name          1088 non-null   object 
dtypes: float64(1), object(7)
memory usage: 68.1+ KB


In [18]:
final['fund_rating'].fillna(np.nan)

0       4.0
1       5.0
2       5.0
3       5.0
4       5.0
       ... 
1083    NaN
1084    NaN
1085    NaN
1086    NaN
1087    NaN
Name: fund_rating, Length: 1088, dtype: float64

In [19]:
for col in ['return_1yr','return_3yr','return_5yr']:
    final[col] = pd.to_numeric(final[col], errors='coerce')

In [20]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1088 entries, 0 to 1087
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Mutual Fund Name  1088 non-null   object 
 1   fund_rating       738 non-null    float64
 2   return_1yr        976 non-null    float64
 3   return_3yr        829 non-null    float64
 4   return_5yr        696 non-null    float64
 5   risk_type         1088 non-null   object 
 6   category          1088 non-null   object 
 7   AMC_name          1088 non-null   object 
dtypes: float64(4), object(4)
memory usage: 68.1+ KB


In [21]:
cols=final.columns.to_list()

In [22]:
print(cols)

['Mutual Fund Name', 'fund_rating', 'return_1yr', 'return_3yr', 'return_5yr', 'risk_type', 'category', 'AMC_name']


In [23]:
cols=['AMC_name','Mutual Fund Name','category','risk_type','fund_rating','return_1yr', 'return_3yr', 'return_5yr']

In [24]:
final=final[cols]

In [25]:
final['risk_type'].unique()

array(['Very High Risk', 'Moderately High Risk', 'High Risk',
       'Moderate Risk', 'Low to Moderate Risk', 'Moderately Low Risk',
       'Low Risk'], dtype=object)

In [26]:
final.to_csv('mutual_funds_india.csv')